In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-extracted-dataset/News_category_extract_dataset.csv


In [2]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [3]:
data = pd.read_csv('../input/news-extracted-dataset/News_category_extract_dataset.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.shape

(70257, 4)

In [4]:
data['category'].value_counts()
data.head()

,category,desc,clean,clean_text
0,POLITICS,Into The Afghan Abyss (Again)-How a failed dru...,"['afghan', 'abyss', 'failed', 'drug', 'war', '...",afghan abyss failed drug war defeat trump afgh...
1,WELLNESS,Exhaustion From Sleep Deprivation: Molecular C...,"['exhaustion', 'sleep', 'deprivation', 'molecu...",exhaustion sleep deprivation molecular clue di...
2,ENTERTAINMENT,Records Aren't Just For Hipsters Anymore-There...,"['record', 'hipster', 'million', 'reason', 'vi...",record hipster million reason vinyl went style...
3,ENTERTAINMENT,Vanity Fair Bloomberg Party: Hollywood And DC ...,"['vanity', 'fair', 'bloomberg', 'party', 'holl...",vanity fair bloomberg party hollywood dc get w...
4,POLITICS,Most Americans Don’t Think The Media Is Honest...,"['american', 'think', 'medium', 'play', 'well'...",american think medium play well trump administ...


In [5]:
le = preprocessing.LabelEncoder()
labels=data['category'].values
labels_clf=le.fit_transform(labels)
classes=le.classes_
print(classes)
d=dict(zip([0,1,2,3],list(classes)))

['ENTERTAINMENT' 'POLITICS' 'TRAVEL' 'WELLNESS']


In [6]:
def clean(text):
    wn=nltk.WordNetLemmatizer()
    stopword=nltk.corpus.stopwords.words('english')
    stopword.extend(['would','could'])
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    tokens=nltk.word_tokenize(text)
    lower=[word.lower() for word in tokens]
    no_stopwords=[word for word in lower if word not in stopword]
    lemm_text=[wn.lemmatize(word) for word in no_stopwords]
    clean_text=lemm_text
    return clean_text

def print_report(pipe):
    y_actuals = y_test
    y_preds_train=pipe.predict(X_train)
    y_preds = pipe.predict(X_test)
    report_train = metrics.classification_report(y_train, y_preds_train)
    report_test = metrics.classification_report(y_actuals, y_preds)
    print("train_accuracy: {:0.3f}".format(metrics.accuracy_score(y_train, y_preds_train)))
    print("test_accuracy: {:0.3f}".format(metrics.accuracy_score(y_actuals, y_preds)))
    print('report_train:\n',report_train)
    print('report_test:\n',report_test)


In [7]:
def metrics_evaluation(y_test, pred):
    cnf_matrix = confusion_matrix(y_test, pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy for each class
    ACC = (TP+TN)/(TP+FP+FN+TN)
    F1= 2*(TPR * PPV) / (TPR + PPV)
    newdf =  np.transpose(pd.DataFrame([TPR,TNR,PPV,NPV,FPR,FNR,ACC,F1],columns = le.classes_))
    n = newdf.columns =['Sensitivity','Specificity','Precision','Negative predictive value','false positive rate','False negative rate','Accuracy','F1 Score']
    #newdf.to_excel("D:\\N.xlsx")
    return newdf

In [8]:
features=data['clean_text']
labels=labels_clf
X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.30,random_state=42)

In [9]:
vec = CountVectorizer()
model_coun_vec = RandomForestClassifier()
pipe1 = make_pipeline(vec, model_coun_vec)
pipe1.fit(X_train,y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [10]:
y_actuals = y_test
#y_preds_train=pipe.predict(X_train)
y_preds = pipe1.predict(X_test)
df1=metrics_evaluation(y_actuals,y_preds)

In [11]:
df1

,Sensitivity,Specificity,Precision,Negative predictive value,false positive rate,False negative rate,Accuracy,F1 Score
ENTERTAINMENT,0.778880,0.957170,0.811467,0.948158,0.042830,0.221120,0.923048,0.794839
POLITICS,0.934105,0.906088,0.875416,0.951135,0.093912,0.065895,0.917687,0.903809
TRAVEL,0.718224,0.986601,0.891159,0.958198,0.013399,0.281776,0.951039,0.795400
WELLNESS,0.870045,0.946184,0.851701,0.953479,0.053816,0.129955,0.926226,0.860775


In [12]:
X_test=np.array(X_test)

In [13]:
import eli5
eli5.show_weights(model_coun_vec, vec=vec, target_names=list(classes), top=15)

Weight,Feature
0.0374 ± 0.0485,trump
0.0108 ± 0.0140,photo
0.0090 ± 0.0097,travel
0.0081 ± 0.0165,donald
0.0081 ± 0.0107,republican
0.0076 ± 0.0108,life
0.0075 ± 0.0117,president
0.0071 ± 0.0106,study
0.0064 ± 0.0096,gop
0.0056 ± 0.0062,movie


In [14]:
idx=3
eli5.show_prediction(model_coun_vec, X_test[idx], vec=vec, target_names=list(classes), top=10)